In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
# CONFIG
OUT_PATH = "../Week3/SnD"

In [4]:
# Load the dataset
df = pd.read_excel('../Week2/SnD/snd.xlsx')

# Change 'Xrock' to 'XROCK' in the 'Offense','Defense', 'FBTeam', 'Winner' columns
df['Offense'] = df['Offense'].replace('Xrock', 'XROCK')
df['Defense'] = df['Defense'].replace('Xrock', 'XROCK')
df['FBTeam'] = df['FBTeam'].replace('Xrock', 'XROCK')
df['Winner'] = df['Winner'].replace('Xrock', 'XROCK')

# Convert blank strings or whitespace-only to NaN
df['PlantSite'] = df['PlantSite'].replace(r'^\s*$', np.nan, regex=True)

# Filter for relevant teams
relevant_teams = ['Q9', 'OUG', 'SPG', 'XROCK', 'GodL', 'Wolves']

df_snd = df[df['Offense'].isin(relevant_teams) | df['Defense'].isin(relevant_teams)]

df_snd.head()

,Date,Map,Offense,Defense,FBTeam,FBPlayer,FBTime,FBWeapon,FBTraded?,PlantSite,PlantClock,Winner,WinType,EndClock,Clutch?,Timeout,DefenseWinner?
0,2025-08-06,Firing Range,Wolves,OUG,OUG,Solo,01:44:00,LMG,No,NaN,NaN,OUG,Elim,00:18:00,NaN,NaN,True
1,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:40:00,Oden,Yes,A,01:10:00,Wolves,Elim,00:06:00,NaN,NaN,False
2,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:48:00,Oden,No,NaN,NaN,Wolves,Elim,01:26:00,NaN,NaN,False
3,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,00:31:00,Oden,Yes,NaN,NaN,OUG,Time,00:00:00,NaN,NaN,True
4,2025-08-06,Firing Range,Wolves,OUG,Wolves,Sound,01:50:00,Nade,No,A,01:28:00,Wolves,Elim,00:16:00,NaN,NaN,False


## SnD Stat 1: First Blood Player Leaderboard

In [6]:
# Count total FBs per player
fb_counts = (
    df_snd.groupby(['FBPlayer', 'FBTeam'])
          .size()
          .reset_index(name='TotalFBs')
)

# Count total rounds played per team
rounds_per_team = (
    pd.concat([
        df_snd.groupby('Offense').size(),
        df_snd.groupby('Defense').size()
    ], axis=1).fillna(0).sum(axis=1).astype(int).reset_index()
)
rounds_per_team.columns = ['FBTeam', 'RoundsPlayed']

# Merge and compute FB rate
fb_leaderboard = (
    fb_counts.merge(rounds_per_team, on='FBTeam', how='left')
             .assign(FBRate=lambda x: x['TotalFBs'] * 100 / x['RoundsPlayed'])
             .sort_values(['FBRate', 'TotalFBs'], ascending=[False, False], ignore_index=True)
)

# Filter for relevant teams
fb_leaderboard = fb_leaderboard[fb_leaderboard['FBTeam'].isin(relevant_teams)].reset_index(drop=True).head(10)
display(fb_leaderboard)
fb_leaderboard.to_csv(f'{OUT_PATH}/fb_leaderboard.csv', index=False)

,FBPlayer,FBTeam,TotalFBs,RoundsPlayed,FBRate
0,Sunuo,XROCK,45,234,19.230769
1,Seven,OUG,52,313,16.613419
2,Abhiz,GodL,30,190,15.789474
3,Maoqi,Q9,35,252,13.888889
4,Bird,SPG,38,279,13.620072
5,Rise,Wolves,33,246,13.414634
6,Zai,Wolves,33,246,13.414634
7,Dchen,Q9,33,252,13.095238
8,Solo,OUG,38,313,12.140575
9,GuXing,SPG,33,279,11.827957


In [7]:
# FBs per team
fb_per_team = (
    df_snd.groupby('FBTeam')
          .size()
          .reset_index(name='TotalFBs')
          .sort_values('TotalFBs', ascending=False)
)
fb_rate_per_team = (
    fb_per_team.merge(rounds_per_team, on='FBTeam', how='left')
    .assign(FBRate=lambda x: x['TotalFBs'] * 100 / x['RoundsPlayed'])
    .sort_values('FBRate', ascending=False, ignore_index=True)
)
# Filter for relevant teams
fb_rate_per_team = fb_rate_per_team[fb_rate_per_team['FBTeam'].isin(relevant_teams)]
display(fb_rate_per_team)

,FBTeam,TotalFBs,RoundsPlayed,FBRate
2,Wolves,132,246,53.658537
3,XROCK,123,234,52.564103
4,SPG,139,279,49.820789
5,Q9,124,252,49.206349
6,OUG,154,313,49.201278
8,GodL,86,190,45.263158


In [8]:
# Compare win rate when drawing first blood vs not drawing first blood
df['FBWin?'] = (df['FBTeam'] == df['Winner']).astype(int)

df['FBWin?'].mean()

0.6836283185840708

In [9]:
# Count plants per map/site (ignore rounds with no plant)
plants = (
    df_snd.dropna(subset=['PlantSite'])
         .groupby(['Map', 'PlantSite'])
         .size()
         .reset_index(name='Plants')
)

# Wide format: columns A and B
site_counts = (
    plants.pivot(index='Map', columns='PlantSite', values='Plants')
          .fillna(0)
          .rename(columns={'A':'Plants_A', 'B':'Plants_B'})
          .reset_index()
)

# Totals and shares
site_counts['TotalPlants'] = site_counts['Plants_A'] + site_counts['Plants_B']
site_counts['Share_A'] = np.where(site_counts['TotalPlants']>0,
                                  site_counts['Plants_A']*100/site_counts['TotalPlants'], 0.0)
site_counts['Share_B'] = np.where(site_counts['TotalPlants']>0,
                                  site_counts['Plants_B']*100/site_counts['TotalPlants'], 0.0)

# Sort and save
site_counts = site_counts.sort_values(['Share_A'], ascending=False, ignore_index=True)
display(site_counts)
site_counts.to_csv(f'{OUT_PATH}/site_counts.csv', index=False)

PlantSite,Map,Plants_A,Plants_B,TotalPlants,Share_A,Share_B
0,Firing Range,104,45,149,69.798658,30.201342
1,Standoff,32,15,47,68.085106,31.914894
2,Tunisia,59,57,116,50.862069,49.137931
3,Coastal,34,38,72,47.222222,52.777778
4,Kurohana,20,27,47,42.553191,57.446809


In [10]:
# Flag whether offense planted
df_snd['Planted'] = pd.notna(df_snd['PlantSite'])

# Flag offense win
df_snd['OffenseWin'] = df_snd['Winner'] == df_snd['Offense']

# Aggregate
off_win_plant_stats = (
    df_snd.groupby(['Offense','Planted'])['OffenseWin']
      .mean()
      .reset_index()
      .pivot(index='Offense', columns='Planted', values='OffenseWin')
      .rename(columns={False:'WinRate_NoPlant', True:'WinRate_Plant'})
      .reset_index()
      .fillna(0)
)

off_win_plant_stats['WinRate_NoPlant'] = off_win_plant_stats['WinRate_NoPlant'] * 100
off_win_plant_stats['WinRate_Plant'] = off_win_plant_stats['WinRate_Plant'] * 100

# Filter for relevant teams
off_win_plant_stats = off_win_plant_stats[off_win_plant_stats['Offense'].isin(relevant_teams)]
display(off_win_plant_stats)

off_win_plant_stats.to_csv(f'{OUT_PATH}/off_win_plant_stats.csv', index=False)

/var/folders/tl/w9hfhthx2q9drpztnsz6w5y80000gn/T/ipykernel_9691/3198823124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snd['Planted'] = pd.notna(df_snd['PlantSite'])
/var/folders/tl/w9hfhthx2q9drpztnsz6w5y80000gn/T/ipykernel_9691/3198823124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snd['OffenseWin'] = df_snd['Winner'] == df_snd['Offense']


Planted,Offense,WinRate_NoPlant,WinRate_Plant
1,GodL,30.612245,67.346939
2,OUG,10.937500,68.478261
3,Q9,15.686275,74.358974
4,SPG,24.074074,74.157303
7,Wolves,19.444444,77.358491
10,XROCK,11.111111,69.387755


In [11]:
len(df_snd[df_snd['Planted'] == True]) / len(df_snd) * 100

52.94840294840295

In [12]:
# Clutch Plays
mask = df_snd['Clutch?'].notna() 
clutch_df = df_snd[mask][['Winner', 'Clutch?']].copy()

# function to parse clutch players
def parse_clutch_players(clutch_str):
    player = clutch_str.split(':')[0].strip()  # Get the player name before the colon
    # if two players, split by '+' and return as a list
    if '+' in player:
        return [p.strip() for p in player.split('+')]
    return [player]

# Apply the function to create a new column with lists of clutch players
clutch_df['ClutchPlayers'] = clutch_df['Clutch?'].apply(parse_clutch_players)

# Explode the DataFrame to have one row per clutch player
clutch_df_exploded = clutch_df.explode('ClutchPlayers')

# Count clutch plays per player
clutch_counts = (
    clutch_df_exploded.groupby('ClutchPlayers')
                      .size()
                      .reset_index(name='ClutchPlays')
                      .sort_values(by='ClutchPlays', ascending=False, ignore_index=True)
                      .head(10)
)

# Rename the column for clarity
clutch_counts.rename(columns={'ClutchPlayers': 'Player'}, inplace=True)

# Display the clutch counts
display(clutch_counts)

# Save clutch counts to CSV
clutch_counts.to_csv(f'{OUT_PATH}/clutch_counts.csv', index=False)

,Player,ClutchPlays
0,JaBen,5
1,Seven,4
2,Abhiz,3
3,Nan,3
4,Wind,3
5,Suiwan,3
6,Bird,3
7,Raph,3
8,Cartels,3
9,Zai,3


In [13]:
# Aggregate plant rate
plant_rate_per_team = (
    df_snd.groupby('Offense')['Planted']
      .mean()
      .reset_index(name='PlantRate')
)

# Aggregate offense win rate
attack_win_rate = (
    df_snd.groupby('Offense')['OffenseWin']
      .mean()
      .reset_index(name='AttackWinRate')
)

# Merge both
plant_rate_per_team = (
    plant_rate_per_team.merge(attack_win_rate, on='Offense')
                       .sort_values('PlantRate', ascending=False)
                       .reset_index(drop=True)
)

plant_rate_per_team['PlantRate'] = plant_rate_per_team['PlantRate'] * 100
plant_rate_per_team['AttackWinRate'] = plant_rate_per_team['AttackWinRate'] * 100

# Filter for relevant teams
plant_rate_per_team = plant_rate_per_team[plant_rate_per_team['Offense'].isin(relevant_teams)]
display(plant_rate_per_team)

,Offense,PlantRate,AttackWinRate
1,SPG,62.237762,55.244755
2,Q9,60.465116,51.162791
3,OUG,58.974359,44.871795
5,GodL,50.000000,48.979592
6,XROCK,43.750000,36.607143
7,Wolves,42.400000,44.000000


In [14]:
# Filter rounds with a plant while team is on defense
retake_df = df_snd[pd.notna(df_snd['PlantSite'])]

# For each defense team: total planted-against rounds
retake_stats = (
    retake_df.groupby('Defense')
             .size()
             .reset_index(name='RoundsWithPlantAgainst')
)

# For each defense team: successful retakes (win by defuse)
retake_success = (
    retake_df[retake_df['WinType'].str.lower() == 'defuse']
        .groupby('Defense')
        .size()
        .reset_index(name='SuccessfulRetakes')
)
# Defense win rate
def_win_rate = (
    df_snd.groupby('Defense')['OffenseWin']
      .apply(lambda x: 100*(1 - x.mean()))
      .reset_index(name='DefenseWinRate')
)
# Merge and compute rate
retake_stats = retake_stats.merge(retake_success, on='Defense', how='left').fillna(0)
retake_stats['RetakeRate'] = retake_stats['SuccessfulRetakes'] *100 / retake_stats['RoundsWithPlantAgainst']

retake_stats = retake_stats.merge(def_win_rate, on='Defense', how='left')
retake_stats = retake_stats[['Defense', 'RetakeRate']]

In [15]:
# Cumulative round differential across ALL rounds
all_rounds = []

# For Offense perspective
offense_results = df_snd[['Offense', 'Winner']].copy()
offense_results['Diff'] = np.where(offense_results['Offense'] == offense_results['Winner'], 1, -1)
offense_results = offense_results.rename(columns={'Offense': 'Team'})[['Team', 'Diff']]
all_rounds.append(offense_results)

# For Defense perspective
defense_results = df_snd[['Defense', 'Winner']].copy()
defense_results['Diff'] = np.where(defense_results['Defense'] == defense_results['Winner'], 1, -1)
defense_results = defense_results.rename(columns={'Defense': 'Team'})[['Team', 'Diff']]
all_rounds.append(defense_results)

# Combine offense + defense
round_diff = (
    pd.concat(all_rounds)
      .groupby('Team')['Diff']
      .sum()
      .rename('RoundDiff')
)

retake_stats = retake_stats.merge(plant_rate_per_team[['Offense', 'PlantRate']], left_on='Defense', right_on='Offense', how='left')

retake_stats = retake_stats.merge(round_diff, left_on='Defense', right_index=True, how='left')


In [16]:
retake_stats = retake_stats[['Defense', 'PlantRate', 'RetakeRate', 'RoundDiff']]
retake_stats = retake_stats.merge(
    fb_rate_per_team[['FBTeam', 'TotalFBs', 'FBRate']],
    left_on='Defense',
    right_on='FBTeam',
    how='left'
)
retake_stats = retake_stats.rename(columns={
    "Defense": "Team",
})
retake_stats = retake_stats[['Team', 'PlantRate', 'RetakeRate', 'RoundDiff', 'FBRate']]

# Filter for relevant teams
retake_stats = retake_stats[retake_stats['Team'].isin(relevant_teams)]

display(retake_stats)


,Team,PlantRate,RetakeRate,RoundDiff,FBRate
1,GodL,50.000000,30.434783,10,45.263158
2,OUG,58.974359,33.333333,-5,49.201278
3,Q9,60.465116,37.735849,38,49.206349
4,SPG,62.237762,18.644068,25,49.820789
7,Wolves,42.400000,33.823529,-2,53.658537
10,XROCK,43.750000,19.718310,-34,52.564103


In [17]:
retake_stats.to_csv(f'{OUT_PATH}/team_stats.csv', index=False)

## Tempo Stats

In [18]:
def parse_clock_to_seconds(x):
    """Parse HH:MM:SS, M:SS, or SS into integer seconds (match time remaining)."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan

    # Handle HH:MM:SS
    parts = s.split(":")
    if len(parts) == 3:  # HH:MM:SS → ignore hours
        mm, ss, _ = parts
        return int(mm) * 60 + int(ss)
    elif len(parts) == 2:  # M:SS
        mm, ss = parts
        return int(mm) * 60 + int(ss)
    elif s.isdigit():  # seconds only
        return int(s)

    return np.nan

def bootstrap_ci_mean(a, n_boot=1000, ci=95, rng=None):
    """Percentile bootstrap CI for the mean; ignores NaNs."""
    arr = pd.Series(a).dropna().to_numpy()
    if arr.size == 0:
        return (np.nan, np.nan)
    if rng is None:
        rng = np.random.default_rng(42)
    boot = np.empty(n_boot, dtype=float)
    n = arr.size
    for i in range(n_boot):
        boot[i] = np.mean(rng.choice(arr, size=n, replace=True))
    alpha = (100 - ci) / 2.0
    return (np.percentile(boot, alpha), np.percentile(boot, 100 - alpha))

# Harmonize FB clock column name
if 'FBClock' not in df.columns and 'FBTime' in df.columns:
    df = df.rename(columns={'FBTime': 'FBClock'})

# Parse clocks -> seconds remaining at the event
for col in ['PlantClock', 'EndClock', 'FBClock']:
    if col in df.columns:
        df[col + '_s'] = df[col].apply(parse_clock_to_seconds)
    else:
        df[col + '_s'] = np.nan  # if missing, fill with NaN

# Calculate elapsed time for planting and end
df['Planted'] = df['PlantSite'].notna()

# Round elapsed
df['RoundElapsed_s'] = np.where(
    df['Planted'],
    (120 - df['PlantClock_s']) + (45 - df['EndClock_s']),
    120 - df['EndClock_s']
)

df['FBElapsed_s'] = 120 - df['FBClock_s']

# Plant elapsed (only if planted)
df['PlantElapsed_s'] = np.where(df['Planted'], 120 - df['PlantClock_s'], np.nan)

# --- Attack-only aggregation with bootstrap CIs ---
rng = np.random.default_rng(42)

def agg_with_ci(group, col):
    mean_val = group[col].mean()
    lo, hi = bootstrap_ci_mean(group[col], n_boot=1000, ci=95, rng=rng)
    return pd.Series({f'{col}_mean': mean_val, f'{col}_CI_low': lo, f'{col}_CI_high': hi})

# Per attacking team
tempo_round = df.groupby('Offense').apply(agg_with_ci, col='RoundElapsed_s').reset_index()
tempo_fb    = df.groupby('Offense').apply(agg_with_ci, col='FBElapsed_s').reset_index()
tempo_plant = (df[df['Planted']]
               .groupby('Offense')
               .apply(agg_with_ci, col='PlantElapsed_s')
               .reset_index())

# Merge all
tempo = (tempo_round
         .merge(tempo_fb, on='Offense', how='left')
         .merge(tempo_plant, on='Offense', how='left')
         .rename(columns={
             'RoundElapsed_s_mean': 'AvgRoundLen_s',
             'FBElapsed_s_mean':    'AvgFBElapsed_s',
             'PlantElapsed_s_mean': 'AvgPlantElapsed_s'
         })
        )

# Optional: order columns nicely
cols_order = [
    'Offense',
    'AvgRoundLen_s','RoundElapsed_s_CI_low','RoundElapsed_s_CI_high',
    'AvgFBElapsed_s','FBElapsed_s_CI_low','FBElapsed_s_CI_high',
    'AvgPlantElapsed_s','PlantElapsed_s_CI_low','PlantElapsed_s_CI_high'
]
tempo = tempo.reindex(columns=[c for c in cols_order if c in tempo.columns])

# Filter for relevant teams
tempo = tempo[tempo['Offense'].isin(relevant_teams)]
display(tempo[['Offense', 'AvgRoundLen_s', 'AvgFBElapsed_s', 'AvgPlantElapsed_s']])

tempo.to_csv(f'{OUT_PATH}/tempo.csv', index=False)

,Offense,AvgRoundLen_s,AvgFBElapsed_s,AvgPlantElapsed_s
2,GodL,66.622449,23.500000,50.000000
3,OUG,76.461538,24.173077,52.978261
4,Q9,76.790698,28.891473,54.576923
5,SPG,85.454545,32.090909,62.101124
8,Wolves,91.128000,32.824000,78.132075
11,XROCK,87.026786,32.303571,66.489796


## Timeout Stats

In [25]:
df_timeouts = pd.read_excel('../Week2/SnD/snd.xlsx', sheet_name='Timeouts')

# --- Total timeouts taken ---
timeouts_per_team = df_timeouts.groupby('TOTeam').size().reset_index(name='TimeoutsTaken')

# --- Unique games played ---
# Each "game" is defined by (Map + Date + Team) combo
games_played = pd.concat([
    df[['Map', 'Date', 'Offense']].rename(columns={'Offense': 'Team'}),
    df[['Map', 'Date', 'Defense']].rename(columns={'Defense': 'Team'})
])

# Drop duplicates so if a team played both O and D on the same map in the same match, it counts as one
games_played = games_played.drop_duplicates(subset=['Map', 'Date', 'Team'])

games_per_team = games_played.groupby('Team').size().reset_index(name='GamesPlayed')

# --- Merge ---
timeouts_stats = timeouts_per_team.merge(games_per_team, left_on='TOTeam', right_on='Team', how='left')
timeouts_stats.drop(columns='Team', inplace=True)
timeouts_stats['TimeoutsPerGame'] = timeouts_stats['TimeoutsTaken'] / timeouts_stats['GamesPlayed']
# Sort
timeouts_stats = timeouts_stats.sort_values(by='TimeoutsTaken', ascending=False)

# Filter for relevant teams
timeouts_stats = timeouts_stats[timeouts_stats['TOTeam'].isin(relevant_teams)]
display(timeouts_stats)
# timeouts_stats.to_csv('../Week2/SnD/timeouts_stats.csv', index=False)

,TOTeam,TimeoutsTaken,GamesPlayed,TimeoutsPerGame
3,OUG,14,19,0.736842
7,Wolves,11,14,0.785714
2,GodL,9,13,0.692308
4,Q9,9,15,0.600000
9,XROCK,6,16,0.375000
5,SPG,5,16,0.312500


In [26]:
# --- Split post-timeout record into individual rounds ---
df_timeouts[['R1', 'R2', 'R3']] = df_timeouts['PostTOStreak'].str.split(',', expand=True)

# Convert to win indicator (1 for W, 0 for L)
for col in ['R1', 'R2', 'R3']:
    df_timeouts[col] = df_timeouts[col].map({'W': 1, 'L': 0})

# --- Bootstrap CIs function ---
def bootstrap_ci(data, n_bootstrap=1000, ci=95):
    stats = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        stats.append(np.mean(sample))
    lower = np.percentile(stats, (100-ci)/2)
    upper = np.percentile(stats, 100 - (100-ci)/2)
    return np.mean(data), lower, upper

# --- Compute win rates + CIs ---
results = []

for team, group in df_timeouts.groupby('TOTeam'):
    # Next round win rate
    mean_nr, low_nr, high_nr = bootstrap_ci(group['R1'].values)

    # 3-round average win rate
    avg_3rounds = group[['R1', 'R2', 'R3']].mean(axis=1).values
    mean_3r, low_3r, high_3r = bootstrap_ci(avg_3rounds)

    results.append({
        'TOTeam': team,
        'TimeoutsTaken': len(group),
        'NextRoundWinRate': mean_nr * 100,
        'NR_CI_Low': low_nr,
        'NR_CI_High': high_nr,
        'ThreeRoundAvgWinRate': mean_3r * 100,
        '3R_CI_Low': low_3r,
        '3R_CI_High': high_3r
    })

timeout_execution_ci = pd.DataFrame(results)

# Filter for relevant teams
timeout_execution_ci = timeout_execution_ci[timeout_execution_ci['TOTeam'].isin(relevant_teams)]
timeout_execution_ci = timeout_execution_ci[['TOTeam', 'TimeoutsTaken', 'NextRoundWinRate', 'ThreeRoundAvgWinRate']]

display(timeout_execution_ci)
timeout_execution_ci.to_csv(f'{OUT_PATH}/timeout_execution.csv', index=False)

,TOTeam,TimeoutsTaken,NextRoundWinRate,ThreeRoundAvgWinRate
2,GodL,9,33.333333,48.148148
3,OUG,14,57.142857,54.761905
4,Q9,9,66.666667,62.962963
5,SPG,5,60.000000,46.666667
7,Wolves,11,36.363636,51.515152
9,XROCK,6,33.333333,44.444444


In [27]:
# MapWin is already stored as "W" or "L" for the team taking the timeout
df_timeouts['MapWin?'] = df_timeouts['MapWin?'].map({'Yes': 1, 'No': 0})

# Group by team and calculate win rate
map_win_rate = (
    df_timeouts.groupby('TOTeam')
    .agg(
        TimeoutsTaken=('MapWin?', 'count'),
        MapWins=('MapWin?', 'sum')
    )
    .reset_index()
)

map_win_rate['MapWinRate'] = map_win_rate['MapWins'] * 100 / map_win_rate['TimeoutsTaken']

# Filter for relevant teams
map_win_rate = map_win_rate[map_win_rate['TOTeam'].isin(relevant_teams)]
display(map_win_rate)
# map_win_rate.to_csv('../Week2/SnD/timeout_map_win_rate.csv', index=False)

,TOTeam,TimeoutsTaken,MapWins,MapWinRate
2,GodL,9,3,33.333333
3,OUG,14,4,28.571429
4,Q9,9,6,66.666667
5,SPG,5,1,20.000000
7,Wolves,11,5,45.454545
9,XROCK,6,1,16.666667


In [29]:
# Merge timeout stats with map win rate
timeouts = timeout_execution_ci.merge(
    map_win_rate[['TOTeam', 'MapWinRate']],
    on='TOTeam',
    how='left'
)
timeouts.to_csv(f'{OUT_PATH}/timeouts.csv', index=False)